## Merge results of ML-Pipeline evaluation on different datasets

### Setup and Helper Functions

#### Imports

In [1]:
from statistics import median_grouped

import pandas as pd
import numpy as np
import shap
from pathlib import Path

from biopsykit.classification.model_selection import SklearnPipelinePermuter

from pepbench.io import compute_mae_std_from_metric_summary

#### Datapaths

In [3]:
model_path_b_point = Path("../../../results/models/b-point")
rater = "rater_01"

### Load B-Point pipeline permuter results

In [4]:
permuter_list = []
permuter_b_point_without_rr_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath(f"without-rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_{rater}.pkl")).metric_summary()
permuter_b_point_without_rr_ms["Dataset"] = "Without-RR-Interval"
permuter_list.append(permuter_b_point_without_rr_ms)
permuter_b_point_without_rr_include_nan_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath(f"without-rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_include_nan_{rater}.pkl")).metric_summary()
permuter_b_point_without_rr_include_nan_ms["Dataset"] = "Without-RR-Interval-Include-Nan"
permuter_b_point_without_rr_include_nan_ms["pipeline_reduce_dim"] = "None"
permuter_b_point_without_rr_include_nan_ms = permuter_b_point_without_rr_include_nan_ms.reset_index().set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf"])
permuter_list.append(permuter_b_point_without_rr_include_nan_ms)
permuter_b_point_without_rr_median_imputed_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath(f"without-rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_median_imputed_{rater}.pkl")).metric_summary()
permuter_b_point_without_rr_median_imputed_ms["Dataset"] = "Without-RR-Interval-Median-Imputed"
permuter_list.append(permuter_b_point_without_rr_median_imputed_ms)
permuter_b_point_rr_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath(f"rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_rr_{rater}.pkl")).metric_summary()
permuter_b_point_rr_ms["Dataset"] = "RR-Interval"
permuter_list.append(permuter_b_point_rr_ms)
permuter_b_point_rr_include_nan_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath(f"rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_rr_include_nan_{rater}.pkl")).metric_summary()
permuter_b_point_rr_include_nan_ms["Dataset"] = "RR-Interval-Include-Nan"
permuter_b_point_rr_include_nan_ms["pipeline_reduce_dim"] = "None"
permuter_b_point_rr_include_nan_ms = permuter_b_point_rr_include_nan_ms.reset_index().set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf"])
permuter_list.append(permuter_b_point_rr_include_nan_ms)
permuter_b_point_rr_median_imputed_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath(f"rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_rr_median_imputed_{rater}.pkl")).metric_summary()
permuter_b_point_rr_median_imputed_ms["Dataset"] = "RR-Interval-Median-Imputed"
permuter_list.append(permuter_b_point_rr_median_imputed_ms)

c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarn

### Merge B-Point pipeline permuter results in one dataframe

In [5]:
merged_permuter = permuter_b_point_without_rr_ms
for p in permuter_list[1:]:
    merged_permuter = pd.concat([merged_permuter, p])
merged_permuter = merged_permuter.reset_index()
merged_permuter = merged_permuter.set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf", "Dataset"])
merged_permuter

conf_matrix  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                  
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval              [0.0]   
                                    RandomForestRegressor Without-RR-Interval              [0.0]   
                                    SVR                   Without-RR-Interval              [0.0]   
                                    KNeighborsRegressor   Without-RR-Interval              [0.0]   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval              [0.0]   
...                                                                                          ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed       [0.0]   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed       [0.0]   
                                    RandomForestRegressor RR-Interval-Median-Imputed       [0.0]   
                                    SVR                   RR-Interval-Median-Imputed       [0.0]   
                                    KNeighborsRegressor   RR-Interval-Median-Imputed       [0.0]   

                                                                                     conf_matrix_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                        
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval                       []   
                                    RandomForestRegressor Without-RR-Interval                       []   
                                    SVR                   Without-RR-Interval                       []   
                                    KNeighborsRegressor   Without-RR-Interval                       []   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval                       []   
...                                                                                                ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed                []   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed                []   
                                    RandomForestRegressor RR-Interval-Median-Imputed                []   
                                    SVR                   RR-Interval-Median-Imputed                []   
                                    KNeighborsRegressor   RR-Interval-Median-Imputed                []   

                                                                                                                            true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                                                         
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    RandomForestRegressor Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    SVR                   Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    KNeighborsRegressor   Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
...                                                                                                                                 ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
               

### Clean merged pipeline permuter

#### Create renaming dict

In [6]:
algo_name_mapping = {
    "StandardScaler": "SS",
    "MinMaxScaler": "MinMax",
    "SelectFromModel": "SFM",
    "SelectKBest": "KBest",
    "DecisionTreeRegressor": "DTR",
    "RandomForestRegressor": "RFR",
    "KNeighborsRegressor": "KNR",
}

#### Apply renaming dict

In [7]:
merged_permuter_renamed = merged_permuter.copy()
merged_permuter_renamed = merged_permuter_renamed.rename(algo_name_mapping)
merged_permuter_renamed

conf_matrix  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                  
SS              SFM                 DTR          Without-RR-Interval              [0.0]   
                                    RFR          Without-RR-Interval              [0.0]   
                                    SVR          Without-RR-Interval              [0.0]   
                                    KNR          Without-RR-Interval              [0.0]   
                KBest               DTR          Without-RR-Interval              [0.0]   
...                                                                                 ...   
MinMax          SFM                 KNR          RR-Interval-Median-Imputed       [0.0]   
                KBest               DTR          RR-Interval-Median-Imputed       [0.0]   
                                    RFR          RR-Interval-Median-Imputed       [0.0]   
                                    SVR          RR-Interval-Median-Imputed       [0.0]   
                                    KNR          RR-Interval-Median-Imputed       [0.0]   

                                                                            conf_matrix_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                        
SS              SFM                 DTR          Without-RR-Interval                       []   
                                    RFR          Without-RR-Interval                       []   
                                    SVR          Without-RR-Interval                       []   
                                    KNR          Without-RR-Interval                       []   
                KBest               DTR          Without-RR-Interval                       []   
...                                                                                       ...   
MinMax          SFM                 KNR          RR-Interval-Median-Imputed                []   
                KBest               DTR          RR-Interval-Median-Imputed                []   
                                    RFR          RR-Interval-Median-Imputed                []   
                                    SVR          RR-Interval-Median-Imputed                []   
                                    KNR          RR-Interval-Median-Imputed                []   

                                                                                                                   true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                                                         
SS              SFM                 DTR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    RFR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    SVR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    KNR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                KBest               DTR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
...                                                                                                                        ...   
MinMax          SFM                 KNR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                KBest               DTR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                                    RFR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                                    SVR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                                    KNR          RR-Interval-Median-Im

#### Introduce unique EstimatorID

In [8]:
merged_permuter_renamed["EstimatorID"] = [f"{scaler}_{reduce_dim}_{reg}_{data}" for scaler, reduce_dim, reg, data in merged_permuter_renamed.index]

In [9]:
# Reset the index to be able to save the dataframe in the .json format
merged_permuter_renamed = merged_permuter_renamed.reset_index()
merged_permuter_renamed

,pipeline_scaler,pipeline_reduce_dim,pipeline_clf,Dataset,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,...,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4,EstimatorID
0,SS,SFM,DTR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[368.5, 352.0, 381.0, 352.0, 337.0, 313.5, 335...","[[368.5, 352.0, 381.0, 352.0, 337.0, 313.5, 33...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",8.233115,0.658923,9.021794,7.630817,9.013441,7.515294,7.984228,SS_SFM_DTR_Without-RR-Interval
1,SS,SFM,RFR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[359.0950787512582, 342.28093771817396, 375.15...","[[359.0950787512582, 342.28093771817396, 375.1...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",7.912126,0.582432,8.439581,7.361962,8.597879,7.119532,8.041678,SS_SFM_RFR_Without-RR-Interval
2,SS,SFM,SVR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[370.2724801709148, 354.33991311082946, 377.49...","[[370.2724801709148, 354.33991311082946, 377.4...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",7.861974,0.838864,8.422173,6.901352,9.110975,7.012360,7.863012,SS_SFM_SVR_Without-RR-Interval
3,SS,SFM,KNR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[358.7954623246977, 356.89675215210076, 370.46...","[[358.7954623246977, 356.89675215210076, 370.4...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",8.675844,0.694709,9.541532,7.904180,9.224159,7.824922,8.884426,SS_SFM_KNR_Without-RR-Interval
4,SS,KBest,DTR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[369.0, 338.0, 378.0, 354.0, 339.0, 316.5, 336...","[[369.0, 338.0, 378.0, 354.0, 339.0, 316.5, 33...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",8.246137,0.649772,8.729208,7.585945,9.266129,7.672706,7.976695,SS_KBest_DTR_Without-RR-Interval
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,MinMax,SFM,KNR,RR-Interval-Median-Imputed,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[522.8348550590839, 517.2146638320321, 487.818...","[[522.8348550590839, 517.2146638320321, 487.81...",...,"[318, 319, 320, 321, 322, 323, 324, 325, 326, ...","[[318, 319, 320, 321, 322, 323, 324, 325, 326,...",9.641310,0.903676,8.651531,9.588186,10.613041,8.639140,10.714651,MinMax_SFM_KNR_RR-Interval-Median-Imputed
68,MinMax,KBest,DTR,RR-Interval-Median-Imputed,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[520.0, 518.0, 497.0, 494.0, 500.0, 339.0, 514...","[[520.0, 518.0, 497.0, 494.0, 500.0, 339.0, 51...",...,"[318, 319, 320, 321, 322, 323, 324, 325, 326, ...","[[318, 319, 320, 321, 322, 323, 324, 325, 326,...",8.806492,1.350393,7.926824,8.905975,9.863095,6.773623,10.562944,MinMax_KBest_DTR_RR-Interval-Median-Imputed
69,MinMax,KBest,RFR,RR-Interval-Median-Imputed,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[523.663160897582, 519.9223176823178, 495.2699...","[[523.663160897582, 519.9

### Save the merged pipeline permuter

In [ ]:
# merged_permuter_renamed.to_json(model_path_b_point.joinpath("permuter_summary/merged_b_point_permuter_paper.json"), orient="records", lines=True)